In [ ]:
#CAROLINA CAMPOS
#ALXANDRA RIVERA

In [1]:
!pip install openai==0.28
!pip install tiktoken

In [4]:
#List of Required Libraries
import openai
import pandas as pd
import numpy as np
from dateutil.parser import parse
from openai.embeddings_utils import get_embedding
from openai.embeddings_utils import distances_from_embeddings
import tiktoken

In [7]:
#Data set importation 
default = pd.read_json('default.jsonl', lines=True)
quest= pd.read_json('queries.jsonl', lines=True)
resp= pd.read_json('corpus.jsonl', lines=True)

In [9]:
#Inspection of datasets
#responsive_direct - df_default
#response_dict['queries']['pages'][0]['extract].split('\n')

df_resp= pd.DataFrame(
    data = resp, 
    columns= ['text']
    
)



#Removal of empty spaces and line skippers
df_resp = df_resp[df_resp.text.str.len()>0] 
df_resp = df_resp['text'].str.replace('\n', '')

df_resp.tail(20)


2028    Scientists have not yet found a way to prevent...
2029    Researchers have not found that eating, diet, ...
2030    - Alagille syndrome is a genetic condition tha...
2031    Too much glucose in the blood for a long time ...
2032    You can do a lot to prevent heart disease and ...
2033    You may have one or more of the following warn...
2034    Narrowed blood vessels leave a smaller opening...
2035    A stroke happens when part of your brain is no...
2036    - Don't smoke.  - Keep blood glucose and blood...
2037    Primary hyperparathyroidism is a disorder of t...
2038    The parathyroid glands are four pea-sized glan...
2039    High PTH levels trigger the bones to release i...
2040    In about 80 percent of people with primary hyp...
2041    Most people with primary hyperparathyroidism h...
2042    Health care providers diagnose primary hyperpa...
2043    Once the diagnosis of primary hyperparathyroid...
2044    Surgery                Surgery to remove the o...
2045    Eating

In [10]:
#Re-organization of responses dataset

df_resp= pd.DataFrame(
    data = df_resp, 
    columns= ['text']
)
df_resp.tail(20)

,text
2028,Scientists have not yet found a way to prevent...
2029,"Researchers have not found that eating, diet, ..."
2030,- Alagille syndrome is a genetic condition tha...
2031,Too much glucose in the blood for a long time ...
2032,You can do a lot to prevent heart disease and ...
2033,You may have one or more of the following warn...
2034,Narrowed blood vessels leave a smaller opening...
2035,A stroke happens when part of your brain is no...
2036,- Don't smoke. - Keep blood glucose and blood...
2037,Primary hyperparathyroidism is a disorder of t...


In [11]:
#Aditional filtering of components in the dataset
            
index_filter = df_resp.apply(lambda x: x.str.startswith('-')).text
df_resp.loc[index_filter, 'text'] = df_resp[index_filter].text.apply(lambda x: x[2:])

df_resp.tail(20)

,text
2028,Scientists have not yet found a way to prevent...
2029,"Researchers have not found that eating, diet, ..."
2030,Alagille syndrome is a genetic condition that ...
2031,Too much glucose in the blood for a long time ...
2032,You can do a lot to prevent heart disease and ...
2033,You may have one or more of the following warn...
2034,Narrowed blood vessels leave a smaller opening...
2035,A stroke happens when part of your brain is no...
2036,Don't smoke. - Keep blood glucose and blood p...
2037,Primary hyperparathyroidism is a disorder of t...


In [12]:
#Saving Responses as cv
df_resp.reset_index(drop=True).to_csv('responses.csv', index=False)

In [13]:
#Inspection of questions dataset

df_quest= pd.DataFrame(
    data = quest, 
    columns= ['text']
    
)

#Removal of empty spaces and line skippers
df_quest = df_quest[df_quest.text.str.len()>0] 
df_quest = df_quest['text'].str.replace('\n', '')

df_quest.tail(20)

2028                   How to prevent Alagille Syndrome ?
2029                   What to do for Alagille Syndrome ?
2030                   What to do for Alagille Syndrome ?
2031    What is (are) Prevent diabetes problems: Keep ...
2032    How to prevent Prevent diabetes problems: Keep...
2033    What are the symptoms of Prevent diabetes prob...
2034    What causes Prevent diabetes problems: Keep yo...
2035    What are the symptoms of Prevent diabetes prob...
2036    How to prevent Prevent diabetes problems: Keep...
2037          What is (are) Primary Hyperparathyroidism ?
2038          What is (are) Primary Hyperparathyroidism ?
2039          What is (are) Primary Hyperparathyroidism ?
2040            What causes Primary Hyperparathyroidism ?
2041    What are the symptoms of Primary Hyperparathyr...
2042        How to diagnose Primary Hyperparathyroidism ?
2043        How to diagnose Primary Hyperparathyroidism ?
2044    What are the treatments for Primary Hyperparat...
2045         W

In [14]:
#Re-organization of responses dataset

df_quest= pd.DataFrame(
    data = df_quest, 
    columns= ['text']
)

df_quest.tail(20)

,text
2028,How to prevent Alagille Syndrome ?
2029,What to do for Alagille Syndrome ?
2030,What to do for Alagille Syndrome ?
2031,What is (are) Prevent diabetes problems: Keep ...
2032,How to prevent Prevent diabetes problems: Keep...
2033,What are the symptoms of Prevent diabetes prob...
2034,What causes Prevent diabetes problems: Keep yo...
2035,What are the symptoms of Prevent diabetes prob...
2036,How to prevent Prevent diabetes problems: Keep...
2037,What is (are) Primary Hyperparathyroidism ?


In [15]:
#Aditional filtering of components in the dataset
            
index_filter = df_quest.apply(lambda x: x.str.endswith('?')).text
df_quest.loc[index_filter, 'text'] = df_quest[index_filter].text.apply(lambda x: x[0:-2])

df_quest.head(20)

,text
0,Who is at risk for Lymphocytic Choriomeningiti...
1,What are the symptoms of Lymphocytic Choriomen...
2,Who is at risk for Lymphocytic Choriomeningiti...
3,How to diagnose Lymphocytic Choriomeningitis (...
4,What are the treatments for Lymphocytic Chorio...
5,How to prevent Lymphocytic Choriomeningitis (LCM)
6,What is (are) Parasites - Cysticercosis
7,Who is at risk for Parasites - Cysticercosis?
8,How to diagnose Parasites - Cysticercosis
9,What are the treatments for Parasites - Cystic...


In [16]:
#Saving Questions as cv
df_quest.reset_index(drop=True).to_csv('questions.csv', index=False)

In [17]:
#Implementation of Open AI

#Creation of embedding

openai.api_key = ''


response = openai.Embedding.create(
    model='text-embedding-ada-002', 
    input=df_resp.text.tolist()
)

question = openai.Embedding.create(
    model='text-embedding-ada-002', 
    input=df_quest.text.tolist()
)

In [18]:
#Verification of the embeddings
print('Responses \n')
response['data'][0]['embedding']

Responses 



[0.006323640234768391,
 -0.012436065822839737,
 0.0034754418302327394,
 -0.023348825052380562,
 -0.027342703193426132,
 0.025755392387509346,
 0.0013392932014539838,
 -0.020161403343081474,
 -0.021607903763651848,
 0.002033741446211934,
 0.011648810468614101,
 0.006115625612437725,
 -0.003760261693969369,
 -0.012276054359972477,
 -0.014516210183501244,
 -0.006784472148865461,
 0.023579241707921028,
 0.022555170580744743,
 0.03013329766690731,
 -0.027035482227802277,
 -0.022350355982780457,
 0.014119382947683334,
 -0.021262280642986298,
 -0.034562405198812485,
 -0.00987588707357645,
 0.011290385387837887,
 0.013914568349719048,
 -0.021070266142487526,
 -0.013504940085113049,
 -0.004115486517548561,
 -0.00037922640331089497,
 -0.0037314596120268106,
 -0.007898150011897087,
 0.0011552803916856647,
 0.006006818264722824,
 0.021441493183374405,
 0.01632113568484783,
 0.0023105607833713293,
 -0.013236121274530888,
 -0.002080144826322794,
 0.016884375363588333,
 0.015015444718301296,
 -0.0104

In [19]:
print('Questions \n')
question_embeddings = question['data'][0]['embedding']
question_embeddings

Questions 



[0.012288585305213928,
 0.015127403661608696,
 -0.003616577247157693,
 -0.02153094857931137,
 -0.024058664217591286,
 0.01865324191749096,
 -0.03372879698872566,
 -0.002636277349665761,
 0.0027594224084168673,
 0.0015976456925272942,
 0.01595701277256012,
 0.030773311853408813,
 -0.03958791121840477,
 0.024266066029667854,
 0.011945074424147606,
 0.010933988727629185,
 0.02789560705423355,
 0.0083673857152462,
 0.0073174117133021355,
 -0.019547663629055023,
 -0.015127403661608696,
 0.0037429628428071737,
 0.013934840448200703,
 -0.02088281698524952,
 0.00138295185752213,
 -0.005865595769137144,
 0.0170588381588459,
 -0.018601391464471817,
 0.01214599609375,
 -0.004076750949025154,
 0.015386655926704407,
 -0.003000851720571518,
 -0.020273571833968163,
 -0.01184785459190607,
 -0.008594231680035591,
 -0.010972877033054829,
 0.024473467841744423,
 0.008127576671540737,
 0.008328497409820557,
 0.005129965487867594,
 0.022179080173373222,
 0.024330878630280495,
 -0.004961451515555382,
 0.005

In [20]:
#Creation of embedding lists and saving as csv

embeddings = list(map(lambda x: x['embedding'], response['data']))
embeddings_q = list(map(lambda x: x['embedding'], question['data']))



df_resp['embeddings'] = embeddings
df_resp.to_csv('embeddings_resp.csv', index=False)


df_quest['embeddings'] = embeddings_q
df_quest.to_csv('embeddings_quest.csv', index=False)

In [21]:
#Importing the response's CSV 

df_responses = pd.read_csv('embeddings_resp.csv')
df_responses['embeddings'] = df_responses['embeddings'].apply(eval).apply(np.array)
#df_questions = pd.read_csv('embeddings_quest.csv')
#df_questions['embeddings_q'] = df_questions['embeddings_q'].apply(eval).apply(np.array)

In [22]:
df_responses

,text,embeddings
0,LCMV infections can occur after exposure to fr...,"[0.006323640234768391, -0.012436065822839737, ..."
1,LCMV is most commonly recognized as causing ne...,"[-0.026094242930412292, -0.0043663159012794495..."
2,Individuals of all ages who come into contact ...,"[0.010534269735217094, -0.012414691969752312, ..."
3,"During the first phase of the disease, the mos...","[-0.0021115741692483425, -0.004429312888532877..."
4,"Aseptic meningitis, encephalitis, or meningoen...","[-0.019981464371085167, 0.015375477261841297, ..."
...,...,...
2043,Once the diagnosis of primary hyperparathyroid...,"[0.011080963537096977, 0.013911773450672626, 0..."
2044,Surgery Surgery to remove the o...,"[0.0014982817228883505, 0.014942212961614132, ..."
2045,"Eating, diet, and nutrition have not been show...","[0.01875200867652893, -0.0011456005740910769, ..."
2046,Primary hyperparathyroidism is a disorder of t...,"[0.013307038694620132, 0.00261545623652637, 0...."


In [23]:
#Finding the cosine distances

distances = distances_from_embeddings(question_embeddings, df_responses['embeddings'].tolist(), distance_metric='cosine')
distances

[0.20505492735481456,
 0.1634145860152474,
 0.1492157879830851,
 0.20851689945114038,
 0.17460441765350732,
 0.1730399690719472,
 0.25004327557950334,
 0.2652451615634386,
 0.2433702657748693,
 0.27459720851532665,
 0.24218963621690348,
 0.2618803191985054,
 0.2649696091885534,
 0.2659665484550736,
 0.2828278693332963,
 0.2726817723070042,
 0.2820531789137588,
 0.2843234343851244,
 0.27996371442818146,
 0.28702995974253687,
 0.27096157337954196,
 0.26724538189041314,
 0.2642030298001502,
 0.264221644490916,
 0.24732834131390113,
 0.22453520883795364,
 0.2544021321926181,
 0.27595571790431694,
 0.2512705744235819,
 0.2739705133811876,
 0.2521997991754962,
 0.2655408787078344,
 0.26254044139529864,
 0.2745654138331235,
 0.2671449578448267,
 0.2546033544068397,
 0.2743424663003453,
 0.28517209615674766,
 0.26964222551486294,
 0.22341510379698282,
 0.2889307630247019,
 0.22660590214414367,
 0.22925431233624471,
 0.2572783238773577,
 0.2542699608255894,
 0.24243190267084447,
 0.222663477577

In [24]:
#Verification and sorting of distances column and saving as csv

df_responses['distances'] = distances

sorted_distances = df_responses.sort_values(by='distances', ascending=True)
sorted_distances.to_csv('distances_sorted.csv', index=False)
sorted_distances

,text,embeddings,distances
2,Individuals of all ages who come into contact ...,"[0.010534269735217094, -0.012414691969752312, ...",0.149216
1,LCMV is most commonly recognized as causing ne...,"[-0.026094242930412292, -0.0043663159012794495...",0.163415
686,Cytomegalovirus (CMV) is a virus found through...,"[-0.005178028251975775, -0.0010677087120711803...",0.172715
5,LCMV infection can be prevented by avoiding co...,"[0.004860565531998873, -0.004707245621830225, ...",0.173040
688,For most people CMV infection is not a problem...,"[-0.0009898442076519132, -0.01749727874994278,...",0.174192
...,...,...,...
1913,Diabetes management and treatment is expensive...,"[0.01575271040201187, -0.020176930353045464, 0...",0.350076
1653,Purpose of Hemodialysis The pur...,"[0.016612136736512184, -0.000823556911200285, ...",0.350202
1839,"Gas is air in the digestive tractthe large, mu...","[0.00881050992757082, 0.024427972733974457, 0....",0.350367
1458,Your doctor can offer you a number of treatmen...,"[-0.01342537347227335, 0.0011852876050397754, ...",0.352561


In [26]:
#Model ft. Custom Text Prompt

In [27]:
#Creation of a sample question 

question= df_quest['text'][0]

In [28]:
#Builting a Custom Text Prompt - Example

tokenizer = tiktoken.get_encoding('cl100k_base')
tokenized = tokenizer.encode(question)
tokenized, len(tokenized)

prompt_template = """
Answer the question based on the context below, and if the question
can't be answered based on the context, say "I don't know"

Context: 

{}

---

Question: {}
Answer:"""

max_token_count = 3000

print(prompt_template.format('context', question))


Answer the question based on the context below, and if the question
can't be answered based on the context, say "I don't know"

Context: 

context

---

Question: Who is at risk for Lymphocytic Choriomeningitis (LCM)?
Answer:


In [29]:
#Let's check how many tokens we have used in this example

tokenized_question = tokenizer.encode(question)
tokenized_prompt = tokenizer.encode(prompt_template)
len(tokenized_question), len(tokenized_prompt)


current_token_count = len(tokenized_question) + len(tokenized_prompt)
current_token_count

59

In [30]:
#Here we are going to provide some context for the sample prompt: 

df = pd.read_csv('distances_sorted.csv')

context = []
current_token_count = len(tokenized_question) + len(tokenized_prompt)

for text in df.text.values:
    text_token_count = len(tokenizer.encode(text))
    try:
        current_token_count += text_token_count
    except:
        print(text_token_count)

    if current_token_count <= max_token_count:
        context.append(text)
    else:
        break
        
df

,text,embeddings,distances
0,Individuals of all ages who come into contact ...,[ 0.01053427 -0.01241469 -0.00591455 ... -0.03...,0.149216
1,LCMV is most commonly recognized as causing ne...,[-0.02609424 -0.00436632 -0.0020876 ... -0.02...,0.163415
2,Cytomegalovirus (CMV) is a virus found through...,[-0.00517803 -0.00106771 -0.00286532 ... -0.02...,0.172715
3,LCMV infection can be prevented by avoiding co...,[ 0.00486057 -0.00470725 0.00655361 ... -0.02...,0.173040
4,For most people CMV infection is not a problem...,[-0.00098984 -0.01749728 0.01601489 ... -0.02...,0.174192
...,...,...,...
2043,Diabetes management and treatment is expensive...,[ 0.01575271 -0.02017693 0.03359914 ... -0.01...,0.350076
2044,Purpose of Hemodialysis The pur...,[ 0.01661214 -0.00082356 0.02796889 ... -0.00...,0.350202
2045,"Gas is air in the digestive tractthe large, mu...",[ 0.00881051 0.02442797 0.01285615 ... 0.00...,0.350367
2046,Your doctor can offer you a number of treatmen...,[-0.01342537 0.00118529 0.03550658 ... 0.00...,0.352561


In [31]:
context


['Individuals of all ages who come into contact with urine, feces, saliva, or blood of wild mice are potentially at risk for infection. Owners of pet mice or hamsters may be at risk for infection if these animals originate from colonies that were contaminated with LCMV, or if their animals are infected from other wild mice. Human fetuses are at risk of acquiring infection vertically from an infected mother.                 Laboratory workers who work with the virus or handle infected animals are also at risk. However, this risk can be minimized by utilizing animals from sources that regularly test for the virus, wearing proper protective laboratory gear, and following appropriate safety precautions.',
 'LCMV is most commonly recognized as causing neurological disease, as its name implies, though infection without symptoms or mild febrile illnesses are more common clinical manifestations.                 For infected persons who do become ill, onset of symptoms usually occurs 8-13 days 

In [32]:
#This allows us to provide context to any of the questions in our dataset, but here we can see which ones have the closest distances to our sample question

print(prompt_template.format('\n\n###\n\n'.join(context), question))


Answer the question based on the context below, and if the question
can't be answered based on the context, say "I don't know"

Context: 

Individuals of all ages who come into contact with urine, feces, saliva, or blood of wild mice are potentially at risk for infection. Owners of pet mice or hamsters may be at risk for infection if these animals originate from colonies that were contaminated with LCMV, or if their animals are infected from other wild mice. Human fetuses are at risk of acquiring infection vertically from an infected mother.                 Laboratory workers who work with the virus or handle infected animals are also at risk. However, this risk can be minimized by utilizing animals from sources that regularly test for the virus, wearing proper protective laboratory gear, and following appropriate safety precautions.

###

LCMV is most commonly recognized as causing neurological disease, as its name implies, though infection without symptoms or mild febrile illnesses 

In [33]:
#Finally, let's send this prompt to a Completion Model. since "text-davinci-003" as been deprecated in the past years, we will be using 'gpt-3.5-turbo-instruct': 

answer = openai.Completion.create(
    model='gpt-3.5-turbo-instruct', 
    prompt=prompt_template.format('\n\n###\n\n'.join(context), question),
    max_tokens = 50
)

In [34]:
print(question, '\n\n', answer['choices'][0]['text'])

Who is at risk for Lymphocytic Choriomeningitis (LCM)? 

  Individuals of all ages who come into contact with urine, feces, saliva, or blood of wild mice are potentially at risk for infection. Owners of pet mice or hamsters, laboratory workers who work with the virus or handle infected animals, and human


In [ ]:
CONTINUACIÓN CHATBOT LAB 6 PARA INTERFACE CON TODOS ESTOS FEATURES COMBINADOS 